## Compte rendu R5.A.10, ANDRIAMISA Nayann, Groupe Nayru

Dans ce fichier, nous verrons un compte rendu de la ressource du BUT informatique (R5.A.10) portant sur les nouvelles bases de données.    
Elle est découpée en 3 grandes parties qui sont respectivement, sur la dénormalisation d'un schéma relationnel de tables et puis d'études des Systèmes de Gestion de Base de Données (SGBD) **Redis** et **MongoDB**. 

## Sommaire :

<ol>
    <li><strong>Dénormalisation</strong></li>
    <ol>
        <li>Dénormalisation d'un schéma relationnel </li>
        <li>Application de la représentation JSON</li>
    </ol>
    <li><strong>Redis</strong></li>
    <ol>
        <li>Bloom filters </li>
        <li>SET_PY</li>
        <li>PUB/SUB</li>
    </ol>
    <li><strong>MongoDB</strong></li>
    <ol>
        <li>Premiers pas avec MongoDB</li>
        <li>Performances avec indexe</li>
        <li>Performances sans indexe</li>
    </ol>
</ol>  

## Dénormalisation :

Il s'agit de faire une sorte de fusion/factorisation d'un ensemble de tables SQL en une seule pour avoir une "structure à plat" afin de minimiser les jointures nécessaires aux opérations.    
Le problème qui se pose est donc de savoir comment passer d'un schéma relationnel en SQL à un modèle NoSQL qui pourrait être implémenté avec nos gestionnaires.    
<br>
Il y a certains critères à respecter dans la fusion dans le cas d'une dénormalisation : 
- Les données qui sont fréquemment intérrogées doivent être rassemblées
- Toutes les données d'un entité doivent être indépendantes
- Une association avec des relations 1+n des deux côtés
- Le taux de mise à jour est le même

Une fois un schéma relationnel dénormalisé, pour être utilisé il convient de l'obetnir sous une représentation JSON.

### <u>Exemple/exercice :</u>

#### 1. Dénormalisation d'un schéma relationnel SQL

Voici un schéma relationnel en SQL, qui a déjà été étudié par les étudiants de 1ère année de BUT informatique pour la ressource ***Introduction aux bases de données et SQL*** :
<br><br>
<code>
**AVIONS** (*NumAv, NomAv, CapAv, VilleAv*);<br>
**PILOTES** (*NumPil, NomPil, NaisPil, VillePil*);<br>
**CLIENTS**(*NumCl, NomCl, NumRueCl, NomRueCl, CodePosteCl, VilleCl*);<br>
**VOLS**(*NumVol, VilleD, VilleA, DateD, HD, DateA, HA, NumPil, NumAv*);<br>
**DEFCLASSES**(*NumVol, Classe, CoeffPrix*);<br>
**RESERVATIONS** (*NumCl, NumVol, Classe, NbPlaces*);<br>
</code>
<br>
Et afin de procéder à une dénormalisation de ce schéma relationnel, nous avons fusionné les tables **VOLS**, **DEFCLASSES** et **RESERVATIONS** par rapport à l'attribut "*NumVol*".    
Voici le résultat obtenu après dénormalisation :

<code>
**AVIONS** (*NumAv, NomAv, CapAv, VilleAv*);<br>
**PILOTES** (*NumPil, NomPil, NaisPil, VillePil*);<br>
**CLIENTS**(*NumCl, NomCl, NumRueCl, NomRueCl, CodePosteCl, VilleCl*);<br>
**VOLS**(*NumVol, VilleD, VilleA, DateD, HD, DateA, HA, NumPil, NumAv, Classe, CoeffPrix, NumCl, NbPlaces*)
</code>

#### 2. Application de la représentation JSON

Nous allons donc commencer par mettre les fichiers dans le bon format, et devoir passer d'un document .txt à un document JSON.

##### Importations et installations

In [ ]:
import pandas as pd
import json

##### On va commencer par les convertir en fichier .csv
En utilisant la fonction <code>to_csv()</code> de la libraire Python : **Pandas**

In [ ]:
avions = pd.read_csv('bddPilotes/AVIONS.txt', sep='\t')
avions.columns = ["NumAv", "NomAv", "CapAv", "VilleAv"]
avions.to_csv('bddPilotes/CSV/AVIONS.csv', index=None)

pilotes = pd.read_csv('bddPilotes/PILOTES.txt', sep='\t')
pilotes.columns = ["NumPil","NomPil","NaisPil","VillePil"]
pilotes.to_csv('bddPilotes/CSV/PILOTES.csv', index=None)

clients = pd.read_csv('bddPilotes/CLIENTS.txt', sep='\t')
clients.columns = ["NumCl", "NomCl", "NumRueCl", "NomRueCl", "CodePosteCl", "VilleCl"]
clients.to_csv('bddPilotes/CSV/CLIENTS.csv', index=None)

vols = pd.read_csv('bddPilotes/VOLS.txt', sep='\t')
vols.columns = ["NumVol","VilleD","VilleA", "DateD","HD","DateA","HA", "NumPil", "NumAv"]
vols.to_csv('bddPilotes/CSV/VOLS.csv', index=None)

defclasses = pd.read_csv('bddPilotes/DEFCLASSES.txt', sep='\t')
defclasses.columns = ["NumVol","Classe", "CoeffPrix"]
defclasses.to_csv('bddPilotes/CSV/DEFCLASSES.csv', index=None)

reservations = pd.read_csv('bddPilotes/RESERVATIONS.txt', sep='\t')
reservations.columns = ["NumCl", "NumVol" ,"Classe", "NbPlaces"]
reservations.to_csv('bddPilotes/CSV/RESERVATIONS.csv', index=None)

Puis nous allons effectuer la jointure sur les classes **VOLS**, **DEFCLASSES** et **RESERVATIONS** en utilisant la fonction <code>merge()</code> pour l'étape de la dénormalisation.    
Et nous finirons par convertir ces fichiers .csv au format JSON grâce à la fonction <code>to_json()</code>

In [ ]:
avions = pd.read_csv('bddPilotes/CSV/AVIONS.csv')
pilotes = pd.read_csv('bddPilotes/CSV/PILOTES.csv')
clients = pd.read_csv('bddPilotes/CSV/CLIENTS.csv')
vols = pd.read_csv('bddPilotes/CSV/VOLS.csv')
defclasses = pd.read_csv('bddPilotes/CSV/DEFCLASSES.csv')
reservations = pd.read_csv('bddPilotes/CSV/RESERVATIONS.csv')

fusion_vols_defclasses = pd.merge(vols, defclasses, how='inner', on=["NumVol"])
fusion_vols_defclasses_reservations = pd.merge(fusion_vols_defclasses, reservations, how='inner', on=["NumVol"])
fusion_vols_defclasses_reservations.to_csv('bddPilotes/CSV/VolsClassesReservations.csv', index=None)

#print(fusion_vols_defclasses_reservations.head(50))

avions.to_json('bddPilotes/JSON/AVIONS.json', orient="records")
pilotes.to_json('bddPilotes/JSON/PILOTES.json', orient="records")
clients.to_json('bddPilotes/JSON/CLIENTS.json', orient="records")
vols.to_json('bddPilotes/JSON/VOLS.json', orient="records")
defclasses.to_json('bddPilotes/JSON/DEFCLASSES.json', orient="records")
reservations.to_json('bddPilotes/JSON/RESERVATIONS.json', orient="records")

fusion = pd.read_csv('bddPilotes/CSV/VolsClassesReservations.csv')
fusion.to_json('bddPilotes/JSON/VolsClassesReservations.json', orient="records")

In [ ]:
with open('bddPilotes/JSON/AVIONS.json', 'r') as file:
    data = json.load(file)
    print(data)

with open('bddPilotes/JSON/PILOTES.json', 'r') as file:
    data = json.load(file)
    print(data)

with open('bddPilotes/JSON/VolsClassesReservations.json', 'r') as file:
    data = json.load(file)
    print(data)

##### Fonction de jointure

In [ ]:
def jointure(fic1, fic2, attribut) :
    mapping_data1 = {item[colonne_jointure]: item for item in data1}

    # Fusionner les données en fonction de la colonne de jointure
    resultat = []
    for item2 in data2:
        item1 = mapping_data1.get(item2[colonne_jointure], {})
        resultat.append({**item1, **item2})

    return resultat

'''
def jointure(json1, json2):

    from json import loads
    from json import dumps

    # First, transform json objects to dictionaries

    d1_name = list(loads(json1))[0]
    #print(d1_name)
    d2_name = list(loads(json2))[0]
    #print(d2_name)

    d1 = loads(json1)[d1_name]
    d2 = loads(json2)[d2_name]

    #print(att_name,type(att_name))
    # Second, iterate through dictionaries
    d_res = {}
    for key1, val1 in d1.items():
        #print(key1, '==', val1)
        for key2, val2 in d2.items():
            #print(key1, '==', key2)
            #print([ord(c) for c in key1],key1,[ord(c) for c in att_name],att_name)
            if key1 == key2:
                d = {}
                d.update(val1)
                d.update(val2)
                #print(d)
                d_res[key1] = d
    my_my_dict = {}
    my_my_dict['test'] = d_res
    z = dumps(my_my_dict)

    return z
'''

## Redis : 

Redis est un SGBD et aussi un gestionnaire **NoSQL** *(Not Only SQL)* qui permet d'adopter le modèle "clé-valeur", ou aussi **JSON**, pour sa structure de données.    
<br>
Afin de se connecter à un serveur Redis depuis ce fichier Jupyter Notebook, pour pouvoir expérimenter ce gestionnaire, j'ai fait appel à ***Docker*** en lancant mon serveur depuis un conteneur avec la commande <code>redis-cli</code>    
<br>

#### <u>Connexion à Redis</u>

##### Les installations et importations nécessaires :

In [ ]:
pip install redis

In [ ]:
import redis
import csv
from json import dumps
import time

client = redis.Redis(host='localhost', port=6380, decode_responses=True)

##### Test de la connection au serveur

In [ ]:
# Ajout de données
client.set("Nom", "Andriamisa")
client.set("Prenom", "Nayann")

# Récupération des données
print(client.get("Nom") + " " + client.get("Prenom"))
client.get("Ressource")

#### <u>Les Bloom filters sur Redis</u>

Le filtre de Bloom est une structure de données probabiliste utilisée pour tester l'appartenance d'un élément à un ensemble. Conçu pour être utilisé dans des situations où la mémoire est limitée et où des faux positifs sont tolérables, le filtre de Bloom permet d'économiser de l'espace en évitant le stockage direct des éléments de l'ensemble. Il utilise plusieurs fonctions de hachage pour attribuer à chaque élément plusieurs positions dans un tableau de bits. Lorsqu'on interroge le filtre pour la présence d'un élément, il renvoie probablement vrai (avec une certaine probabilité d'erreur), indiquant ainsi que l'élément pourrait être dans l'ensemble, ou certainement faux s'il n'est pas présent. Redis offre une implémentation efficace du filtre de Bloom, permettant son utilisation dans un contexte de base de données à mémoire vive avec des avantages significatifs en termes d'optimisation de l'utilisation de la mémoire.

##### 1. Le code python implémentant les Blooms filters

In [ ]:
def perf_bloom(csv_file, n) :

    client.delete("bloom")
    client.execute_command("BF.RESERVE", "bloom", "0.01", "1700000")

    with open(csv_file, encoding='utf-8') as csvfile:
        my_reader = csv.DictReader(csvfile, delimiter='\t')
        my_data = [my_row for my_row in my_reader]

        pres = dup = 0
        print('Start to create the bloom filter over', n, 'inputs')
        st = time.process_time()

        for my_row in my_data[0:n]:
            if not client.execute_command("BF.EXISTS", "bloom", my_row['M']):
                client.execute_command("BF.ADD", "bloom", my_row['M'])
                pres = pres + 1
            else:
                dup = dup + 1

        et = time.process_time()
        res = et - st
        print('CPU Execution time:', res, 'seconds')
        print('We found', dup, 'duplicates in the input')

# Step 1
perf_bloom("DEMO.csv", 14000)

##### 2. Set.py pour des test de performances

Voici une fonction qui teste les performances d'ajout d'éléments distincts du fichier CSV à un ensemble Redis.

In [ ]:
def perf_set(csv_file, n):

    r = redis.Redis(host='localhost', port=6380, decode_responses=True)

    r.delete("my_set")

    with open(csv_file, encoding = 'utf-8') as csvfile:
        my_reader = csv.DictReader(csvfile,delimiter='\t')
        my_data = [my_row for my_row in my_reader]
        #print(my_data)
        pres = 0
        dup = 0
        print('Start to create the set over',n,'inputs')
        # get the start time
        st = time.process_time()
        for my_row in my_data[0:n]:
            #print(my_row['M'])
            if not r.sismember("my_set", my_row['M']):
                r.sadd("my_set", my_row['M'])
                pres += 1
            else:
                dup += 1
        # get the end time
        et = time.process_time()
        # get execution time
        res = et - st
        print('CPU Execution time:', res, 'seconds')
        print('We found',dup,'duplicates in the input')
        print()
        print('Wall time (also known as clock time or wall-clock time) is simply the total time')
        print('elapsed during the measurement. It\'s the time you can measure with a stopwatch.')
        print('It is the difference between the time at which a program finished its execution and')
        print('the time at which the program started. It also includes waiting time for resources.')
        print()
        print('CPU Time, on the other hand, refers to the time the CPU was busy processing')
        print('the program\'s instructions. The time spent waiting for other task to complete')
        print('(like I/O operations) is not included in the CPU time. It does not include')
        print('the waiting time for resources.')
#Step 1
 
perf_set("DEMO.csv", 1400)

### PUB/SUB avec Redis

### Les installations et importations utilisées pour tester PUB/SUB avec Redis

In [ ]:
pip install schedule

In [ ]:
import redis

import json
import random
from datetime import datetime
import time
import schedule

client = redis.Redis(host="localhost", port=6380, decode_responses=True)

#### Publisher

Dans le code ci-dessous, la fonction <code>publish()</code> va publier un message qui sera un couple d'une **date** (à l'occurence la date du jour ainsi que l'heure) et d'une **CO2_value** (un nombre aléatoire entre 300 et 1000).    
Ces messages seront publiés de manière espacée entre 1 et 3 secondes (aléatoirement aussi).  
<br>  
*Afin que la fonction PUB/SUB de Redis fonctionne correctemment, il faut éxécuter le publisher indépendamment du subscriber (dans un autre jupyter avec un noyau différent de celui utilisé dans ce compte rendu par exemple).*

In [ ]:
def generate_random_co2_value():
    return round(random.uniform(300, 1000))

while True:
    current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    co2_value = generate_random_co2_value()
    
    # Créer un dictionnaire
    data = {'date': current_date, 'co2_value': co2_value}
    
    # Convertir le dictionnaire en format JSON
    json_data = json.dumps(data)
    
    client.publish('testPubSub', json_data)

    #print(r.get('testPubSub'))

    time.sleep(random.uniform(1, 3))

#### Subscriber

Dans le code ci-dessous, la fonction <code>pubsub()</code> va définir le comportement du client et la fonction <code>psubscribe()</code> va dire que le client connecté est "abonné" aux messages publiés dans la chaîne "testPubSub".    
Et elle executera toutes les minutes un calcul de la moyenne des valeurs de CO2 que le client a reçu dans la dernière minute.
<br>  

In [ ]:
def func():
    global my_list
    #
    # Calcul de la moyenne
    #
    if len(my_list) > 0:
    	print("La moyenne des valeurs de CO2 sur la dernière minute est:", sum(my_list) / len(my_list))
    my_list.clear()

schedule.every(10).seconds.do(func)

p = client.pubsub()
p.psubscribe('testPubSub')

my_list = []

first_message = True

while True:
    schedule.run_pending()
    m = p.get_message()
    #schedule.run_pending()
    if m != None:
       if not first_message:
          print('Message reçu:', m['data'])
          data_str = m['data']
          #data_str = m['data'].decode('utf-8')  # Décoder les bytes en str
          data_dict = json.loads(data_str)  # Charger la chaîne JSON en un dictionnaire Python
          co2_value = data_dict.get('co2_value')
          if co2_value is not None:
                my_list.append(co2_value)

       else:
        print("Erreur : ça marche pas")
        first_message = False
        time.sleep(1)

## MongoDB

MongoDB est un SGBD et aussi un gestionnaire **NoSQL** *(Not Only SQL)* qui permet d'adopter le modèle  **JSON** pour sa structure de données.  
<br>
Afin de se connecter à un serveur MongoDB depuis ce fichier Jupyter Notebook, pour pouvoir expérimenter ce gestionnaire, j'ai fait appel à ***Docker*** en lancant mon serveur depuis un conteneur avec la commande <code>mongosh</code>    
<br>

#### Les installations et importations nécessaires pour l'expérimentation de MongoDB

In [ ]:
pip install pymongo

In [ ]:
import pymongo

connection = pymongo.MongoClient(host="localhost", port=27017)

#### Premiers pas avec MongoDB

In [ ]:
db = connection["db_R5A10"]

collection = db["test_bd_jupyter"]

# Ajout
data = {"nom":"Andriamisa", "Prenom":"Nayann"}
collection.insert_one(data)
data = {"Date":"22-12-2023", "Jour":"Vendredi"}
collection.insert_one(data)


# Modification
collection.update_one({"Date":"22-12-2023"},{"$set":{"Date":"23-12-2023"}})

# Suppression
collection.delete_many({"Date":"23-12-2023"})

# Consultation
for doc in collection.find() :
    print(doc)
